yfinance API Documentation: https://ranaroussi.github.io/yfinance/reference/index.html 

git repo: https://github.com/tomwmoore/tm-finance-lab

In [ ]:
import sys 
import os

import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime

import importlib

# Add root path so other subfolders are accessible
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))


# Database Functions
import src.utils.db_azure as db
importlib.reload(db) # Reload it every time the cell is run

# Custom Plots
import src.utils.plots
importlib.reload(src.utils.plots) # Force reload every time the cell is run
from src.utils.plots import plot_stock

# Custom Indicators
import src.utils.indicators
importlib.reload(src.utils.indicators) # Force reload every time the cell is run
from src.utils.indicators import compute_rsi


# Helper Functions
import src.utils.yfinance_helpers
importlib.reload(src.utils.yfinance_helpers) # 

<module 'src.utils.yfinance_helpers' from 'c:\\Users\\tomwm\\Documents\\Data_Science\\tm_finance_lab\\src\\utils\\yfinance_helpers.py'>

# Plot Stock Price and Indicators

In [42]:
# get stock data from postgres db
symbol = 'CJ.TO'
azure_engine = db.get_analytics_azure_engine()
select_query = f"select * from stock_prices where symbol = '{symbol}'"

df = pd.read_sql(select_query,azure_engine)

df.head()

,date,symbol,open,high,low,close,volume,updated_at
0,2020-01-02,CJ.TO,1.850790,1.871983,1.808406,1.822534,267000,2025-10-08 22:14:16
1,2020-01-03,CJ.TO,1.871982,1.886110,1.836661,1.850790,395400,2025-10-08 22:14:16
2,2020-01-06,CJ.TO,1.864918,1.992072,1.861386,1.985008,791800,2025-10-08 22:14:16
3,2020-01-07,CJ.TO,1.977944,2.020329,1.956752,1.999136,556500,2025-10-08 22:14:16
4,2020-01-08,CJ.TO,2.013264,2.055649,1.879047,1.956751,768200,2025-10-08 22:14:16


In [43]:
# get WTI data and merge in with stock prices
select_query = f"select * from stock_prices where symbol = 'CL=F'"
df_oil = pd.read_sql(select_query,azure_engine) 

df_oil = df_oil.rename(columns={'close': 'wti_price_close'})

df = df.merge(df_oil[['date','wti_price_close']], on = ['date'], how ='left')

In [ ]:
# Plot prices & custom indicators
import src.utils.plots
importlib.reload(src.utils.plots) # Reload it every time the cell is run

from src.utils.plots import plot_stock

import src.utils.indicators as ind
importlib.reload(ind) # Reload it every time the cell is run


df_plot = df.copy()

target_period = 15

df_plot = df_plot[df_plot['date'] > '2025-01-01']

df_plot['rolling_avg'] = ind.compute_rolling_avg(df_plot['close'],period = target_period)

upper_band, lower_band = ind.compute_bollinger_bands(df_plot['close'],period = target_period)
df_plot['upper_band'] = upper_band
df_plot['lower_band'] = lower_band

df_plot['rsi'] = ind.compute_rsi(df_plot['close'],period=target_period)

# normalize the data
df_plot['close'] = df_plot['close']/df_plot['close'].iloc[0]
df_plot['rolling_avg'] = df_plot['rolling_avg']/df_plot[~df_plot['rolling_avg'].isna()]['rolling_avg'].iloc[0]
df_plot['upper_band'] = df_plot['upper_band']/df_plot[~df_plot['upper_band'].isna()]['upper_band'].iloc[0]
df_plot['lower_band'] = df_plot['lower_band']/df_plot[~df_plot['lower_band'].isna()]['lower_band'].iloc[0]
df_plot['wti_price_close'] = df_plot['wti_price_close']/df_plot['wti_price_close'].iloc[0]
 
fig = plot_stock(df_plot, 
                 date_col='date', 
                 price_col='close',
                 rsi = df_plot['rsi'],
                 plot_volume= True,
                 indicators= {
                    f'Rolling Avg ({target_period}d)': {'data': df_plot['rolling_avg'], 'color': 'red'},
                    'Upper Band': {'data': df_plot['upper_band'], 'color': 'green', 'dash': 'dot'},
                    'Lower Band': {'data': df_plot['lower_band'], 'color': 'green', 'dash': 'dot'},
                    'WTI Front Month': {'data': df_plot['wti_price_close'], 'color': 'purple', 'dash': 'dot'}
                })
fig.show()

In [35]:

df_plot = df_oil[df_oil['date']> '2025-01-01']

fig = plot_stock(df_plot, 
                 date_col='date', 
                 price_col='wti_price_close',
                 rsi = None,
                 plot_volume= True)
fig.show()

# Feature Pipeline

In [84]:
df_ppl = df_plot.copy()

df_ppl.reset_index(inplace=True)

In [85]:

importlib.reload(src.pipelines.features)
from src.pipelines.features import FeaturePipeline


ppl = FeaturePipeline(df_ppl)

In [86]:
ppl.run_pipeline()



,index,date,symbol,open,high,low,close,volume,updated_at,wti_price_close,rolling_avg,upper_band,lower_band,rsi,rsi_14,bb_upper_14,bb_lower_14
0,1255,2025-01-02,CJ.TO,6.011005,6.167974,6.011005,1.000000,965700,2025-10-08 22:14:16,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1256,2025-01-03,CJ.TO,6.158741,6.251076,6.158741,1.013513,1130400,2025-10-08 22:14:16,1.011350,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1257,2025-01-06,CJ.TO,6.269543,6.315711,6.218759,1.012012,622900,2025-10-08 22:14:16,1.005880,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1258,2025-01-07,CJ.TO,6.260309,6.306476,6.223375,1.024024,444900,2025-10-08 22:14:16,1.015315,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1259,2025-01-08,CJ.TO,6.315710,6.320326,6.232608,1.016516,697500,2025-10-08 22:14:16,1.002598,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,1455,2025-10-20,CJ.TO,7.550000,7.710000,7.550000,1.234245,579800,2025-10-26 16:57:49,0.786545,1.270047,1.313731,1.223978,41.552507,42.723008,1.364560,1.199873
201,1456,2025-10-21,CJ.TO,7.650000,7.660000,7.550000,1.237497,518000,2025-10-26 16:57:49,0.790647,1.266934,1.313865,1.217441,43.255817,46.500004,1.365656,1.195525
202,1457,2025-10-22,CJ.TO,7.650000,7.825000,7.650000,1.268394,581200,2025-10-26 16:57:49,0.799945,1.267471,1.313975,1.218427,51.141557,49.047620,1.365396,1.194855
203,1458,2025-10-23,CJ.TO,7.980000,8.030000,7.875000,1.281403,588900,2025-10-26 16:57:49,0.844934,1.267900,1.314353,1.218911,50.917430,53.623190,1.366094,1.197642
